In [1]:
print("OK!")

OK!


In [3]:

from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers


c:\Users\mohit\anaconda3\envs\chatbot\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
PINECONE_API_KEY = "784c0e1f-3e6b-435a-841a-2389c2c0c84c"
PINECONE_API_ENV = "gcp-starter"

In [4]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [5]:
extracted_data = load_pdf("data/")

In [6]:
# extracted_data

In [7]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [8]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 60447


In [9]:
# text_chunks

In [10]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [11]:
embeddings = download_hugging_face_embeddings()

c:\Users\mohit\anaconda3\envs\chatbot\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\mohit\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [12]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [13]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [14]:
#query_result

In [15]:
!pip install pinecone-client

In [16]:
!pip show pinecone-client

Name: pinecone-client
Version: 3.2.0
Summary: Pinecone client and SDK
Home-page: https://www.pinecone.io
Author: Pinecone Systems, Inc.
Author-email: support@pinecone.io
License: Apache-2.0
Location: C:\Users\mohit\anaconda3\envs\chatbot\Lib\site-packages
Requires: certifi, tqdm, typing-extensions, urllib3
Required-by: 


In [19]:
import pinecone
import os

# Set your Pinecone API key
os.environ["PINECONE_API_KEY"] = "784c0e1f-3e6b-435a-841a-2389c2c0c84c"

# Initialize Pinecone
pc = pinecone.Pinecone()

index_name = "medical"

# Assuming you have defined huggingface_embeddings and text_chunks
docsearch=Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

# Now create or update the index



In [20]:
#If we already have an index we can load it like this
docsearch=Pinecone.from_existing_index(index_name, embeddings)

query = "What are Allergies"

docs=docsearch.similarity_search(query, k=3)

print("Result", docs)

Result [Document(page_content='hobbies, recreational inhalants) exposures. Allergen-sensitized\npatients may complain of symptoms on exposure to known allergens\nsuch as animals and may complain of increased symptoms during\nspecific pollen seasons. Up to two-thirds of patients with asthma will\nbe atopic (as opposed to half of the U.S. population), and almost half\nwill have a history of rhinitis, with many complaining of intermittent\nsinusitis. In patients with adult-onset asthma, a careful occupational'), Document(page_content='eustachian tubes precipitates secondary infections of the sinuses\nand middle ear , respectively . A growing number of patients with\nseasonal allergic rhinitis demonstrate pollen-associated food\nallergen syndrome characterized by oropharyngeal pruritus and/or\nmild swelling following the ingestion of plant-based foods in the\nsame plant family as a tree, grass, or weed, which contain cross-\nreacting allergens.\nThe nose presents a large mucosal surface ar

In [1]:
prompt_template="""
context: What is the patient's medical history?
question: What are the patient's current symptoms?
question: Would you like to know whom to consult? (Please respond with yes or no)
context: Symptoms: <current_symptoms>
question: Would you like to know about specialists for these symptoms? (Please respond with yes or no)
"""

In [4]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [23]:
llm=CTransformers(model="llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [24]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [25]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])

c:\Users\mohit\anaconda3\envs\chatbot\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Response :  Acne is a chronic inflammatory skin condition characterized by comedones (blackheads and whiteheads), pimples, and other lesions on the face, scalp, and body. It is caused by a complex interplay of endogenous and exogenous factors, including genetic predisposition, hormonal imbalances, environmental stressors, and microbial dysbiosis. Treatment typically involves topical or systemic medications that target the various components of the acne pathway, such as comedone formation, sebum production, inflammation, and bacterial load.


KeyboardInterrupt: 